# Function Call - Google Custom Search API 版

# 準備

In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
import openai
from googleapiclient.discovery import build

# 環境変数の読み込み
load_dotenv()
openai.api_key = os.environ['API_KEY']
model_name = "gpt-3.5-turbo"
google_cse_id = os.environ['GOOGLE_CSE_ID']
google_api_key = os.environ['GOOGLE_API_KEY']

# Google検索API

In [40]:
import json

# ニュースを返す関数の作成
def get_search_result(keyword, when, unit="d"):
    service = build("customsearch", "v1", developerKey=google_api_key)
    result = service.cse().list(q=keyword, dateRestrict=f"{when}{unit}", cx=google_cse_id).execute()

    # 検索結果のトップ3を配列に取得
    result_list = []
    for i, item in enumerate(result["items"]):
        result_list.append({"title": f'{item["title"]} {item["snippet"]}'})
        if i>=2:
            break

    return json.dumps({"result": result_list})

In [42]:
# テスト用コード
ret = get_search_result("東京駅のイベント", 1, "m")
ret2 = json.loads(ret)
for n in ret2["result"]:
    print("-"*20)
    print(n["title"])

--------------------
東京駅周辺・丸の内でおすすめのイベント｜レッツエンジョイ東京 東京丸の内の丸ビルレストラン7店舗にて「＜ニコラ・フィアット＞ 七夕レストランフェア」が開催されます。 2023/07/01(土) ～ 07/15(土); 丸ビル. 東京駅.
--------------------
東京駅(東京都)周辺のイベント｜ウォーカープラス 東京駅(東京都)周辺で開催されるイベント情報15件をお届けします。今日開催されているイベントはもちろん、週末の「どこ行こう」に役立つ情報が満載！
--------------------
東京キャラクターストリート特設 ｜ 東京駅一番街 ストリート内のイベントスペース「いちばんプラザ」では、期間限定のキャラクターショップやイベントを実施しており、いつでも新鮮さを感じていただけます。


In [44]:
# 関数情報の作成
functions=[
    {
        "name": "get_search_result",
        "description": "指定したキーワードの検索結果を取得する",
        "parameters": {
            "type": "object",
            "properties": {
                "keyword": {
                    "type": "string",
                    "description": "キーワード",
                },
                "when": {
                    "type": "number",
                    "description": "日付や時間の範囲"
                },
                "unit": {
                    "type": "string",
                    "enum": ["d", "h", "y"]
                },
            },
            "required": ["keyword"],
        },
    }
]

In [45]:
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = openai.ChatCompletion.create(
    model=model_name,
    messages=[
        {"role": "user", "content": question},
    ],
    functions=functions,
    function_call="auto",
)

In [46]:
response

<OpenAIObject chat.completion id=chatcmpl-7XgQ037HnhnlacPi170gKf9VRdK5t at 0x2115554af30> JSON: {
  "id": "chatcmpl-7XgQ037HnhnlacPi170gKf9VRdK5t",
  "object": "chat.completion",
  "created": 1688262236,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_search_result",
          "arguments": "{\n  \"keyword\": \"\u6771\u4eac\u99c5\u306e\u30a4\u30d9\u30f3\u30c8\",\n  \"when\": 1,\n  \"unit\": \"m\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 135,
    "completion_tokens": 40,
    "total_tokens": 175
  }
}

In [47]:
response["choices"][0]["message"]["function_call"]["arguments"]

'{\n  "keyword": "東京駅のイベント",\n  "when": 1,\n  "unit": "m"\n}'

In [48]:
# 言語モデルからの回答メッセージを取得
message = response["choices"][0]["message"]

# モデルが関数呼出と判断した
if message.get("function_call"):
    # 関数名の取得
    function_name = message["function_call"]["name"]
    print(f"関数名：{function_name}")
    # 引数の取得
    arguments = json.loads(message["function_call"]["arguments"])
    print(f"引数：{arguments}")

    # 関数名で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    print(f"関数の実行結果：{function_response}")

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response2 = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0]["message"]["content"].strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print(response.choices[0]["message"]["content"].strip())

関数名：get_search_result
引数：{'keyword': '東京駅のイベント', 'when': 1, 'unit': 'm'}
関数の実行結果：{"result": [{"title": "\u6771\u4eac\u99c5\u5468\u8fba\u30fb\u4e38\u306e\u5185\u3067\u304a\u3059\u3059\u3081\u306e\u30a4\u30d9\u30f3\u30c8\uff5c\u30ec\u30c3\u30c4\u30a8\u30f3\u30b8\u30e7\u30a4\u6771\u4eac \u6771\u4eac\u4e38\u306e\u5185\u306e\u4e38\u30d3\u30eb\u30ec\u30b9\u30c8\u30e9\u30f37\u5e97\u8217\u306b\u3066\u300c\uff1c\u30cb\u30b3\u30e9\u30fb\u30d5\u30a3\u30a2\u30c3\u30c8\uff1e \u4e03\u5915\u30ec\u30b9\u30c8\u30e9\u30f3\u30d5\u30a7\u30a2\u300d\u304c\u958b\u50ac\u3055\u308c\u307e\u3059\u3002 2023/07/01(\u571f) \uff5e 07/15(\u571f); \u4e38\u30d3\u30eb. \u6771\u4eac\u99c5."}, {"title": "\u6771\u4eac\u99c5(\u6771\u4eac\u90fd)\u5468\u8fba\u306e\u30a4\u30d9\u30f3\u30c8\uff5c\u30a6\u30a9\u30fc\u30ab\u30fc\u30d7\u30e9\u30b9 \u6771\u4eac\u99c5(\u6771\u4eac\u90fd)\u5468\u8fba\u3067\u958b\u50ac\u3055\u308c\u308b\u30a4\u30d9\u30f3\u30c8\u60c5\u583115\u4ef6\u3092\u304a\u5c4a\u3051\u3057\u307e\u3059\u3002\u4eca\u65

# 複数のFunctionの切り替え

In [56]:
import json
import requests
from bs4 import BeautifulSoup

# ニュースを検索する関数
def get_news(keyword, when, unit="d"):
    # WebサイトのURLを指定
    url = f'https://news.google.com/search?q={keyword} when:{when}{unit}&hl=ja&gl=JP&ceid=JP:ja'

    # Requestsを利用してWebページを取得する
    response = requests.get(url)
    html = response.text

    # BeautifulSoupを利用してWebページを解析する
    soup = BeautifulSoup(html, 'html.parser')

    # soup.selectを利用して、ニュースのタイトルを取得する
    elements = soup.select('article h3 a')

    # ニュースのトップ3を配列に取得
    news = []
    for i, element in enumerate(elements):
        news.append({"title": element.getText()})
        if i>=2:
            break

    return json.dumps({"news": news})

In [57]:
# テスト用コード
ret = get_news("夏祭り", 5)
ret2 = json.loads(ret)
for n in ret2["news"]:
    print("-"*20)
    print(n["title"])

--------------------
「Wヒーロー夏祭り2023」 登場キャラ・イベントスケジュール解禁 ...
--------------------
関西の夏祭りトップ切り「愛染祭」フランスの女性、チャンチキの ...
--------------------
大人みこし豪快練り歩き 奈良・天川の天水分神社4年ぶり夏祭り ...


In [58]:
# 関数情報の作成
functions=[
    {
        "name": "get_search_result",
        "description": "指定したキーワードの検索結果を取得する",
        "parameters": {
            "type": "object",
            "properties": {
                "keyword": {
                    "type": "string",
                    "description": "キーワード",
                },
                "when": {
                    "type": "number",
                    "description": "日付や時間の範囲"
                },
                "unit": {
                    "type": "string",
                    "enum": ["d", "h", "y"]
                },
            },
            "required": ["keyword"],
        },
    },
    {
        "name": "get_news",
        "description": "指定したキーワードのニュースを取得する",
        "parameters": {
            "type": "object",
            "properties": {
                "keyword": {
                    "type": "string",
                    "description": "キーワード",
                },
                "when": {
                    "type": "number",
                    "description": "日付や時間の範囲"
                },
                "unit": {
                    "type": "string",
                    "enum": ["d", "h", "y"]
                },
            },
            "required": ["keyword"],
        },
    }
]

In [63]:
# question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
question = "夏祭りに関する5日以内のニュースを教えてください"

response = openai.ChatCompletion.create(
    model=model_name,
    messages=[
        {"role": "user", "content": question},
    ],
    functions=functions,
    function_call="auto",
)

In [64]:
response

<OpenAIObject chat.completion id=chatcmpl-7XgTjBjbregV6GRCVfdhSJEOmSFDQ at 0x21156883a70> JSON: {
  "id": "chatcmpl-7XgTjBjbregV6GRCVfdhSJEOmSFDQ",
  "object": "chat.completion",
  "created": 1688262467,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_news",
          "arguments": "{\n  \"keyword\": \"\u590f\u796d\u308a\",\n  \"when\": 5,\n  \"unit\": \"d\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 188,
    "completion_tokens": 33,
    "total_tokens": 221
  }
}

In [65]:
response["choices"][0]["message"]["function_call"]["arguments"]

'{\n  "keyword": "夏祭り",\n  "when": 5,\n  "unit": "d"\n}'

In [66]:
# 言語モデルからの回答メッセージを取得
message = response["choices"][0]["message"]

# モデルが関数呼出と判断した
if message.get("function_call"):
    # 関数名の取得
    function_name = message["function_call"]["name"]
    # 引数の取得
    arguments = json.loads(message["function_call"]["arguments"])

    # 関数名で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    elif function_name == "get_news":
        function_response = get_news(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response2  = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2 .choices[0]["message"]["content"].strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print(response.choices[0]["message"]["content"].strip())

以下は関連する5日以内の夏祭りに関するニュースです：

1. ウィヒロ夏祭り2023、登場キャラ・イベントスケジュール解禁など (元記事: 「Wヒーロー夏祭り2023」登場キャラ・イベントスケジュール解禁…)

2. 関西の夏祭りトップス「愛染祭」チャンピオンの女性、チャンチャキの…

3. 大人みこし送り歩き ブルーム・天川の天水分神社4年ぶり夏祭り…

これらのニュースは、夏祭りに関する情報とイベントの予定、さらには個別の夏祭りのトピックに焦点を当てています。
